# Tutorial 05, Part 1: Pandas DataFrames 
[The official project homepage](https://pandas.pydata.org)

* Goal - extend what we learned about Series objects in the previous tutorial to their 2D counterpart - DataFrames
    * Take this chance to also learn a bit about file input/output (I/O) and some other more advanced coding techniques

## DataFrames

[Pandas quick start guide for DataFrames](https://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe)

* A DataFrame (DF) is a labeled data struture that can be thought of as a 2D extension of the Series objects that we discussed in the first part of the tutorial
* A DF can accept many types of input, from a 2D ndarray, multiple Series, a dict of 1D arrays, another DF, etc
* Like a Series, DFs contain data values and their labels. Because we're now dealing with a 2D structure, we call the row labels the index argument and the column labels the column argument. 
    * Like a Series, if you don't explicitly assign row and column labels, then they will be auto-generated (but not as useful as specifying the labels yourself!)

<div class="alert alert-info">
Much of what we learned about Series objects will generalize to DFs, so here we'll focus on some of key functionality that might not be obvious based on the previous tutorial.
</div>

<div class="alert alert-info">
One more quick note: if using an older version of Python (earlier than 3.6) and Pandas (earlier than 0.23) and you create a DF from a dict without explicitly specifying column names, then the column names will be entered into the DF based on lexical order
</div>

## Import libs

In [1]:
# standard numpy and matplotlib imports
import numpy as np
import matplotlib as plt

# for plotting in a separte window (not inline with notebook output)
# %matplotlib qt

# import a generic pandas object and also a few specific functions that we'll use
import pandas as pd 
from pandas import DataFrame, read_csv

# new - get and store current file path for file i/o later on in tutorial
import os
cwd = os.getcwd()

# also define the default font we'll use for figures. 
fig_font = {'fontname':'Arial', 'size':'20'}

## Make up a data set to demonstrate functionality, will import some real data later on
* Here we'll pretend that we did a unit recording experiment 
    * There are two stimulus conditions
    * And we are recording from 10 different neurons 

In [2]:
# seed random number generator so that we're all seeing the same thing in class
np.random.RandomState(0)

In [56]:
# index lables for our 10 neurons...see previous tutorial for more elegant ways of generating
# index labels, here we're just going to write them out
neuron_labels = ['Nrn0', 'Nrn1','Nrn2','Nrn3','Nrn4','Nrn5','Nrn6','Nrn7','Nrn8','Nrn9']  

In [57]:
# generate response in each neuron to stimulus 1...
resp1_hz = [14, 27, 62, 88, 45, 56, 75, 63, 33, 46]

In [58]:
# generate a response to stimulus 2...use random.randint just for practice/fun
min_resp = 0  # inclusive
max_resp = 90 # exclusive
resp2_hz = np.random.randint(min_resp, max_resp, len(resp1_hz))

## New - use 'zip' function to wrap up the data from each list into one list
[reference page for zip](https://www.w3schools.com/python/ref_func_zip.asp)

* Operates just like it sounds  - takes a set of iterators and groups them together into a single iterator with the 1st element in the resultant iterator comprised of the first element of each iterator 'zipped' together, then the second element from each iterator zipped together, etc. 
* Length of resulting iterator limited by the length of the shortest input iterator!

<div class="alert alert-warning">
Because the length of the resulting iterator is limited by length of shortest input iterator, you can sometimes not get an error if you try to zip together iterators with unequal lengths - this is fine if intenitonal, but if the unequal length was caused by a bug, then you may not find it when using zip!
</div>

In [59]:
neuron_data = list(zip(resp1_hz, resp2_hz))
print(neuron_data)

print('Grab one index to see the two response arrays zipped together:')
print(neuron_data[9])

[(14, 87), (27, 80), (62, 61), (88, 13), (45, 55), (56, 64), (75, 27), (63, 14), (33, 85), (46, 69)]
Grab one index to see the two response arrays zipped together:
(46, 69)


## Make a DataFrame object to hold the contents of the data set
[DataFrame help page](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.html)

* Just like with the pd.Series call, you specify the data, index labels (row labels in this case)
* In addition to row labels, you can also specify column labels (with 'columns')
* Can also specify data type (default is inferred)
* Like pd.Series you can ask for an independent copy of the data (copy=True) or you will get a view by default (i.e. copy=False)

In [61]:
# make the call to pd.DataFrames to create the DF - usage much like pd.Series
df = pd.DataFrame(data = neuron_data, index=neuron_labels, columns = ['resp1', 'resp2'])

# take a look at the output...
display(df)   # compare to print(df) - looks nicer with display thanks to iPython backend 

,resp1,resp2
Nrn0,14,87
Nrn1,27,80
Nrn2,62,61
Nrn3,88,13
Nrn4,45,55
Nrn5,56,64
Nrn6,75,27
Nrn7,63,14
Nrn8,33,85
Nrn9,46,69


## Get a high-level summary of the data using built-in functionality of DataFrame object
[API reference page](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.describe.html)

In [69]:
# first call this using the defaults
df.describe()

,resp1,resp2
count,10.000000,10.00000
mean,50.900000,55.50000
std,22.521348,28.06441
min,14.000000,13.00000
25%,36.000000,34.00000
50%,51.000000,62.50000
75%,62.750000,77.25000
max,88.000000,87.00000


In [72]:
# can specify different values to change behavior...
df.describe(percentiles=np.linspace(0,1,11))

,resp1,resp2
count,10.000000,10.00000
mean,50.900000,55.50000
std,22.521348,28.06441
min,14.000000,13.00000
0%,14.000000,13.00000
10%,25.700000,13.90000
20%,31.800000,24.40000
30.0%,41.400000,46.60000
40%,45.600000,58.60000
50%,51.000000,62.50000


<div class="alert alert-warning">

BEGIN SIDEBAR

Important bit of info for avoiding a common source of confusion (and potential bugs!!!)


* Note that if you make a DF out of a set of Series (e.g. a dict of Series), then the resulting DF index labels will be the union of the index labels in each Series
* This can be confusing because the DF will still be formed even if you have mismatching labels or even if you have two series of different sizes..
* Fortunately, the misaligned (or missing) values will be filled in with NaNs ('Not-a-Number') to serve as a placeholder for the misaligned or missing info
* Quick demo below before continuing on with our sample neuron data from above 
</div>

In [45]:
# make a set of two Series with unequal lengths stored in a dict, 
# with each Series having data and index labels

# Note that Series 1 has 4 elements, but Series 2 has 5 elements!

data_dict = {'dict0' : pd.Series(data = np.random.randn(4), index=['0','1','2','3']), 
            'dict1' : pd.Series(data = np.random.randint(0,5,5), index=['0','1','2','3','4'])}

# make a data frame
weird_df = pd.DataFrame(data_dict)

# take a look - notice that pd.DataFrame did not throw an error even though
# the input Series are different sizes...however, it did mark the missing value 
# with a NaN
display(weird_df)

,dict0,dict1
0,-0.623741,2
1,-0.231649,1
2,0.433477,3
3,-0.136111,3
4,NaN,4


<div class = "alert alert-important">
Because of the above behavior, often good to frequently check for NaNs in your data to identify processing steps that might not have gone awry...(unless you are expecting NaNs as part of routine processing, in which case this might not be very helpful). 
Many ways to do this, but here is one way that works pretty well with identifying the presence of a NaN anywhere in the DataFrame (which is handy beacuse often times the DataFrames are too large to easily see in their entirety) 
</div>

In [49]:
#show True/False for each element in DF
display(np.isnan(weird_df))

# use the 'any' method to figure out if any entries are NaN
if np.isnan(weird_df).any:
    print('weird...you have NaNs in your data')

,dict0,dict1
0,False,False
1,False,False
2,False,False
3,False,False
4,True,False


weird...you have NaNs in your data


<div class="alert alert-warning">
END SIDEBAR...now back to the main thread and our 10 neuron experimental data
</div>

## Indexing, adding, deleting entire columns
* Think of the DF as a dict of Series objects with common labels - much of the syntax is the same as for dicts (and for Series)

In [86]:
# grab the second column from our DF
display(df['resp2'])

Nrn0    87
Nrn1    80
Nrn2    61
Nrn3    13
Nrn4    55
Nrn5    64
Nrn6    27
Nrn7    14
Nrn8    85
Nrn9    69
Name: resp2, dtype: int64

2nd column, 2nd entry
80
2nd column, 4th entry
13


### Adding a column is easy and can be done dynamically (on the fly)

In [96]:
# define a third response column as the product of the first two columns
df['resp3'] = df.resp1 * df.resp2
display(df)

,resp1,resp2,resp3
Nrn0,14,87,1218
Nrn1,27,80,2160
Nrn2,62,61,3782
Nrn3,88,13,1144
Nrn4,45,55,2475
Nrn5,56,64,3584
Nrn6,75,27,2025
Nrn7,63,14,882
Nrn8,33,85,2805
Nrn9,46,69,3174


### Removing columns is also easy and done on the fly... 

In [97]:
# using the del command will delete a column from the DF
# note that here you have to use the df['resp3'] notation
# the df.resp3 notation will not work.
del df['resp3']

### Instead of deleting outright, you can also "pop" a column out and asign it to another variable

In [100]:
# define a third response column as the product of the first two columns
df['resp3'] = df.resp1 * df.resp2

# then pop it out
resp3 = df.pop('resp3')
display(resp3)

# now df is back down to just 2 columns
print('\n')
display(df)

Nrn0    1218
Nrn1    2160
Nrn2    3782
Nrn3    1144
Nrn4    2475
Nrn5    3584
Nrn6    2025
Nrn7     882
Nrn8    2805
Nrn9    3174
Name: resp3, dtype: int64

,resp1,resp2
Nrn0,14,87
Nrn1,27,80
Nrn2,62,61
Nrn3,88,13
Nrn4,45,55
Nrn5,56,64
Nrn6,75,27
Nrn7,63,14
Nrn8,33,85
Nrn9,46,69


## More on indexing and selection of specific coordinates in a DF

### Row selection - this is a bit more confusing sometimes
* You can use df.loc to select a row by its label name
* You can use df.iloc to select a row by its integer location 
* You can use boolean vectors to select a set of rows that satisfy some condition
* You can slice rows using standard notation e.g. df[1:3] for rows 1-3

<div class="alert alert-danger">
Contrary to usual slicing conventions, both the start and the stop indices are included when using the DF.LOC function...see below for demo. This makes sense (sort of) because you're indexin by label name, not by a zero-based integer index. 
</div>

In [103]:
# data from 2nd neuron across both stimulus conditions
df.loc['Nrn1']

resp1    27
resp2    80
Name: Nrn1, dtype: int64

In [108]:
# CAREFUL!
# data from 2nd-6th neuron inclusive across both stimulus conditions
df.loc['Nrn1':'Nrn5']

# again, just need to be careful but this makes sense given that you're indexing based on label name (not 0-based counting)

,resp1,resp2
Nrn1,27,80
Nrn2,62,61
Nrn3,88,13
Nrn4,45,55
Nrn5,56,64


In [105]:
# data from 5th neuron across both stimulus conditions
df.iloc[4]

resp1    45
resp2    55
Name: Nrn4, dtype: int64

In [106]:
# data from 2nd-5th neuron across both stimulus conditions
df[2:5]

,resp1,resp2
Nrn2,62,61
Nrn3,88,13
Nrn4,45,55


In [ ]:
# can use the trick for returning only a subset of values from a function that we discussed in
# the randomization/bootstrapping lecture:
# here grab the 2nd entry from the 2nd column
print('2nd column, 2nd entry')
print(df['resp2'][1])

# can also go like this
print('2nd column, 4th entry')
print(df.resp2[3])

## Data alignment - like with the Series object, but accounts for a union of both column and index(row) labels
* Recall from the tutorial on Series objects that if you have mismatched or different length Series, you can still combine them and do operations on them.
* The missing/mismatched elements in the resulting Series was marked with a NaN, and the same will happen with a DF

In [111]:
# make two dfs and then combine them...

# first df will be 3 x 4
r0 = 3
c0 = 4

df0 = pd.DataFrame(np.random.rand(r0,c0), index=['r1', 'r2', 'r3'], columns=['a', 'b', 'c', 'd'])

# second df will be 4 x 5
r1 = 4
c1 = 5

df1 = pd.DataFrame(np.random.rand(r1,c1), index=['r1', 'r2', 'r3','r4'], columns=['a', 'b', 'c', 'd','e'])

# now combine using a unary operation (lets do element by element multiplication)

df_prod = df0 * df1

# note the NaNs in all the places where there is a mismatch between the two original dfs
display(df_prod)

,a,b,c,d,e
r1,0.116048,0.785854,0.215822,0.753481,NaN
r2,0.556773,0.211081,0.033690,0.005973,NaN
r3,0.269004,0.836581,0.089661,0.068497,NaN
r4,NaN,NaN,NaN,NaN,NaN


## Similar to above example, but now make two dfs that are the same size, but only a subset of the labels match
* To demonstrate that combining dfs relies on a union of both row (index) and column labels, have one of each mismatch 
* Write this as if we made a common coding error and mislabeled some of index/column labels

In [114]:
# first df will be 4 x 4
r0 = 4
c0 = 4

df0 = pd.DataFrame(np.random.rand(r0,c0), index=['r1', 'r2', 'rr3','r4'], columns=['a', 'b', 'c', 'd'])

# second df will be 4 x 5
r1 = 4
c1 = 4

df1 = pd.DataFrame(np.random.rand(r1,c1), index=['r1', 'r2', 'r3','r4'], columns=['a', 'b', 'cv', 'd'])

# now combine using a unary operation (lets do element by element multiplication)

df_prod = df0 * df1

# note the NaNs in all the places where there is a mismatch 
# between the index/column labels in two original dfs
display(df_prod)

,a,b,c,cv,d
r1,0.466101,0.253740,NaN,NaN,0.088492
r2,0.012832,0.003991,NaN,NaN,0.006623
r3,NaN,NaN,NaN,NaN,NaN
r4,0.022493,0.258495,NaN,NaN,0.072945
rr3,NaN,NaN,NaN,NaN,NaN


## Just as with the Series objects, we can use most NumPy operations on DFs without any trouble 

In [115]:
# true matrix mulitplication (not element by element)
np.dot(df0, df1)

array([[ 1.43215958,  1.30659757,  1.44269617,  0.93094285],
       [ 0.32398104,  0.63098308,  0.36154496,  0.23244512],
       [ 0.35722631,  0.39662632,  0.35717389,  0.25713967],
       [ 0.35454342,  0.16776709,  0.81952309,  0.31967657]])

## Get a high-level summary of the data using built-in functionality of DataFrame object
[API reference page](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.describe.html)

In [69]:
# first call this using the defaults
df.describe()

,resp1,resp2
count,10.000000,10.00000
mean,50.900000,55.50000
std,22.521348,28.06441
min,14.000000,13.00000
25%,36.000000,34.00000
50%,51.000000,62.50000
75%,62.750000,77.25000
max,88.000000,87.00000


In [72]:
# can specify different values to change behavior...
df.describe(percentiles=np.linspace(0,1,11))

,resp1,resp2
count,10.000000,10.00000
mean,50.900000,55.50000
std,22.521348,28.06441
min,14.000000,13.00000
0%,14.000000,13.00000
10%,25.700000,13.90000
20%,31.800000,24.40000
30.0%,41.400000,46.60000
40%,45.600000,58.60000
50%,51.000000,62.50000


## And also just like the Series object, you can also apply many other standard analyses using the DF object directly

* [Pandas doc for all functions](https://pandas.pydata.org/pandas-docs/stable/api.html#api-dataframe-stats)

In [ ]:
df.mean(axis=0)

## Given our main DF with the data from the simulated unit recording study, demonstrate some simple file I/O
* Start by saving the data in a csv (comma separated values) file, which is a common format that is useful for small-to-medium sized data and can easily store mixed data types (strs, ints, floats, etc)

[more than you ever wanted to know about the CSV format](https://www.loc.gov/preservation/digital/formats/fdd/fdd000323.shtml)

[API reference page for csv read/write in Pandas](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html)

In [63]:
# Check out API reference page for all the flags, but for the moment
# the we'll save our header in the file so that reading it back in is easier...
# can do this either way, you just need to know how a file was written when 
# go to read it 
df.to_csv('spike_rates.csv',index=True,header=True)

### Have a look at the csv file in a text editor...

In [67]:
# use our current working directory to build a path to the file
print(cwd)
file_name = cwd + '/spike_rates.csv'
print(file_name)

# read back in the .csv file that we made above. 
# by default read_csv will try to infer the column headers but since
# we explicitly wrote them out (header=True on write) we can tell 
# read_csv that row 0 in the file has the header info...
# same for the row labels (index labels)...tell
# read_csv that our row labels are in the first column (0) 
# of the file
df = pd.read_csv(file_name, index_col=0, header=0)
df

C:\Users\jserences\Documents\GitHub\dev
C:\Users\jserences\Documents\GitHub\dev/spike_rates.csv


,resp1,resp2
Nrn0,14,87
Nrn1,27,80
Nrn2,62,61
Nrn3,88,13
Nrn4,45,55
Nrn5,56,64
Nrn6,75,27
Nrn7,63,14
Nrn8,33,85
Nrn9,46,69


## Making cooler DataFrame styles (and more useful...although that should take a backseat to coolness)
[Check here for a bunch of neat style options](https://pandas.pydata.org/pandas-docs/stable/style.html)
* Simple demo - can write custom functions that highlight specific aspects of your data - can be very useful for more clearly highlighting/communicating key points in the data within a notebook  

In [ ]:
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]